In [23]:
import os
import sys
import pandas as pd

groupid = 'phenotype'
method = 'wilcox.test'
group_file = 'merged_input.group_info.tsv'
group = pd.read_csv(group_file, sep='\t', index_col=0)
phenos_df = pd.read_csv('merged_input.group_info.tsv', sep='\t', index_col=0, header=0)
phenos = list(set(phenos_df[groupid]))
g1 = phenos[0]
g2 = phenos[1]

col_rename = {
    'mean({})'.format(g1): 'g1_mean',
    'mean({})'.format(g2): 'g2_mean',
    'sd({})'.format(g1): 'g1_variance',
    'sd({})'.format(g2): 'g2_variance',
    'occ-rate({})'.format(g1): 'g1_occ',
    'occ-rate({})'.format(g2): 'g2_occ',
    'occ-n({})'.format(g1): 'g1_n',
    'occ-n({})'.format(g2): 'g2_n',
    'pvalue': 'p_value',
    'qvalue': 'p_adj',
}
taxonomy_fullname = 'kingdom/phylum/class/order/family/genus/species'
fullname_dict = {}
for t in taxonomy_fullname.split('/'):
    fullname_dict[t[0]] = t

p_df_new = pd.DataFrame(columns=(['taxonomy_level', 'group1', 'group2']+list(col_rename.values())+['g1/g2', 'enriched']))
p_pass_df_new = pd.DataFrame(columns=(['taxonomy_level', 'group1', 'group2']+list(col_rename.values())+['g1/g2', 'enriched']))
abd_df_pass_new = pd.DataFrame(columns=phenos_df.index)
for tax, tax_fullname in fullname_dict.items():
    output_p = 'output.diff_testing.'+ method + '.' + tax + '.stat.tsv'
    output_pass_a = 'output.diff_testing.'+ method + '.' + tax + '.abd.pass.tsv'
    abd_df_pass = pd.read_csv(output_pass_a, sep='\t', index_col=0)
    p_df = pd.read_csv(output_p, sep='\t', index_col=0)
    p_df.rename(columns=col_rename, inplace=True)
    p_df['taxonomy_level'] = tax_fullname
    p_df['g1/g2'] = p_df['g1_mean']/p_df['g2_mean']
    p_df['g1_variance'] = p_df['g1_variance']**2
    p_df['g2_variance'] = p_df['g2_variance']**2
    p_df['group1'] = g1
    p_df['group2'] = g2
    for t in p_df.index:
        p_df_new.loc[t, :] = p_df.loc[t, list(p_df_new.columns)]
        if t in abd_df_pass.index:
            p_pass_df_new.loc[t, :] = p_df_new.loc[t, :]
            abd_df_pass_new.loc[t, ] = abd_df_pass.loc[t, abd_df_pass.columns]
p_df_new.to_csv('output.diff_testing.'+ method + '.tsv', sep='\t')
p_pass_df_new.to_csv('output.diff_testing.'+ method + '.pass.tsv', sep='\t')
abd_df_pass_new.to_csv('output.diff_testing.'+ method + '.pass.abundance.tsv', sep='\t')
    
    